In [47]:
# https://www.kaggle.com/fedesoriano/heart-failure-prediction

import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier

from sklearn.preprocessing import MinMaxScaler
from category_encoders import OneHotEncoder

from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.metrics import f1_score, make_scorer
from sklearn.model_selection import GridSearchCV

data = pd.read_csv("heart.csv")

data

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,0,Normal,172,N,0.0,Up,0
1,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat,1
2,37,M,ATA,130,283,0,ST,98,N,0.0,Up,0
3,48,F,ASY,138,214,0,Normal,108,Y,1.5,Flat,1
4,54,M,NAP,150,195,0,Normal,122,N,0.0,Up,0
...,...,...,...,...,...,...,...,...,...,...,...,...
913,45,M,TA,110,264,0,Normal,132,N,1.2,Flat,1
914,68,M,ASY,144,193,1,Normal,141,N,3.4,Flat,1
915,57,M,ASY,130,131,0,Normal,115,Y,1.2,Flat,1
916,57,F,ATA,130,236,0,LVH,174,N,0.0,Flat,1


Tento dataset obsahuje následující sloupce:

Age - věk pacienta
Sex - pohlaví pacienta
ChestPainType - Typ bolesti na hrudi [TA, ATA, NAP, ASY] - typical, atypical (spojeno se srdečním svalem), non-anginal (bolest na hrudi, která nesouvisí se srdečním svalem) asymptotic - žádná bolest
RestingBP - Klidový krevní tlak 
Cholesterol - hladina cholesterolu 
FastingBS - hladina cukru na lačno (1 pokud je víc než 120 jinak 0)
RestingECG - EKG v klidu [Normal, ST, LVH] normal - normální, ST - abnormalita, LVH - pravděpodobně nebo definitivně hypertrofie (zvětšení) levé komory
MaxHR - Maximální tep. frekvence (60 - 202)
ExerciseAngina - Jestli je bolest vyvolaná při cvičení (Y - ano, N - ne)
Oldpeak - Hodnota jak moc je hodnota ST na grafu oproti baseline (pokud je záporná, tak jde pod ní, pokud je kladná, tak je nad ní)
ST_Slope - [Up, Flat, Down] - Stoupající, neměnící se, klesající 
HeartDisease: [0, 1] - 0 nemá, 1 má

Záznamy pocházejí z různých klinik a celkově je záznamů 918. Našim cílem je naučit náš program na základě všech sloupců předpovědět, jestli dotyčný trpí nebo netrpí srdeční chorobou.

Nejdřív se podíváme jak vypadají jednotlivé sloupce a jestli chybí hodnoty

In [48]:
print(data.isnull().sum())

print(data.describe())

Age               0
Sex               0
ChestPainType     0
RestingBP         0
Cholesterol       0
FastingBS         0
RestingECG        0
MaxHR             0
ExerciseAngina    0
Oldpeak           0
ST_Slope          0
HeartDisease      0
dtype: int64
              Age   RestingBP  Cholesterol   FastingBS       MaxHR  \
count  918.000000  918.000000   918.000000  918.000000  918.000000   
mean    53.510893  132.396514   198.799564    0.233115  136.809368   
std      9.432617   18.514154   109.384145    0.423046   25.460334   
min     28.000000    0.000000     0.000000    0.000000   60.000000   
25%     47.000000  120.000000   173.250000    0.000000  120.000000   
50%     54.000000  130.000000   223.000000    0.000000  138.000000   
75%     60.000000  140.000000   267.000000    0.000000  156.000000   
max     77.000000  200.000000   603.000000    1.000000  202.000000   

          Oldpeak  HeartDisease  
count  918.000000    918.000000  
mean     0.887364      0.553377  
std      1.066

Žádné hodnoty nechybí, tak se pustíme do úpravy datasetu

Age - Použiju min-max scaler pro hezčí rozdělení hodnot
Sex - tam použiju one hot encoding 
ChestPainType - tam použiju OneHotEncoding proto, abych se vyvaroval tomu dávat pořadí a 4 stringové typy potřebujeme převést na čiselnou reprezentaci
RestingBP - pro jistotu bych použil min-max scaler, abychom měli jistotu, že to bude hezky rozprostřené
cholesterol - to samé, min-max scaler
FastingBS - tam máme rovnou bool hodnoty, není co měnit
RestingECG - OneHotEncoding ze stejného důvodu co ChestPainType
MaxHR - min-max scaler
ExerciseAngina - tam bych převedl Y-N na 1 a 0 ručně pomocí list comprehension
Oldpeak - tam bych použil min-max scaler, aby hodnoty byly hezčí, aktuálně na 1. pohled mají velké rozsahy
ST_Slope - OneHotEncoding
HeartDisease - není co měnit, výsledný sloupec s bool hodnotama


In [49]:

data["ExerciseAngina"] = [0 if x == "N" else 1 for x in data["ExerciseAngina"]]

hot_encoder = OneHotEncoder(cols=["Sex", "ChestPainType", "RestingECG", "ST_Slope"])
data = hot_encoder.fit_transform(data)

min_max_scaler = MinMaxScaler()
data[["Age", "RestingBP", "MaxHR", "Cholesterol", "Oldpeak"]] = min_max_scaler.fit_transform(
    data[["Age", "RestingBP", "MaxHR", "Cholesterol", "Oldpeak"]])

data

,Age,Sex_1,Sex_2,ChestPainType_1,ChestPainType_2,ChestPainType_3,ChestPainType_4,RestingBP,Cholesterol,FastingBS,RestingECG_1,RestingECG_2,RestingECG_3,MaxHR,ExerciseAngina,Oldpeak,ST_Slope_1,ST_Slope_2,ST_Slope_3,HeartDisease
0,0.244898,1,0,1,0,0,0,0.70,0.479270,0,1,0,0,0.788732,0,0.295455,1,0,0,0
1,0.428571,0,1,0,1,0,0,0.80,0.298507,0,1,0,0,0.676056,0,0.409091,0,1,0,1
2,0.183673,1,0,1,0,0,0,0.65,0.469320,0,0,1,0,0.267606,0,0.295455,1,0,0,0
3,0.408163,0,1,0,0,1,0,0.69,0.354892,0,1,0,0,0.338028,1,0.465909,0,1,0,1
4,0.530612,1,0,0,1,0,0,0.75,0.323383,0,1,0,0,0.436620,0,0.295455,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
913,0.346939,1,0,0,0,0,1,0.55,0.437811,0,1,0,0,0.507042,0,0.431818,0,1,0,1
914,0.816327,1,0,0,0,1,0,0.72,0.320066,1,1,0,0,0.570423,0,0.681818,0,1,0,1
915,0.591837,1,0,0,0,1,0,0.65,0.217247,0,1,0,0,0.387324,1,0.431818,0,1,0,1
916,0.591837,0,1,1,0,0,0,0.65,0.391376,0,0,0,1,0.802817,0,0.295455,0,1,0,1


Nyní vidíme, že všechny hodnoty byly převedeny pomocí one hot encodingu nebo min max scaleru a můžeme se pustit na výběr našeho měření úspěšnosti

In [50]:
print(data["HeartDisease"].value_counts(normalize=True))

HeartDisease
1    0.553377
0    0.446623
Name: proportion, dtype: float64


Jak je vidět, dataset je celkem hezky rozložený. Přesto bych ale zvolil F1 metriku, která je komplexnější než např. jen accuracy, ale lépe nám dokáže měřit falešně pozitivní (takže řekneme, že má nemoc, i když nemá) a falešně negativní (řekneme, že pacient je zdravý, i když není)

Jdeme se vrhnout na první učení a to za použití rozhodovacího stromu, ale nejdřív si ještě rozdělíme data na testovací a vybereme sloupce na trénování, přičemž vytvoříme testovací data 40% z původních (60/40)

In [51]:
X = data.drop("HeartDisease", axis=1)
y = data["HeartDisease"]

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=25)

In [52]:
tree_classifier = DecisionTreeClassifier(random_state=25)
tree_classifier.fit(x_train, y_train)

y_pred = tree_classifier.predict(x_test)

print(f"Přesnost F1: {round(f1_score(y_test, y_pred), 3) * 100}%")

Přesnost F1: 82.1%


Bez ladících parametrů máme 82% úspěšnost, to není špatné, ale věřím, že to jde i lépe. 

Jako 2. algoritmus zkusíme RandomForest

In [59]:
random_classifier = RandomForestClassifier(random_state=25)
random_classifier.fit(x_train, y_train)

y_pred = random_classifier.predict(x_test)

print(f"Přesnost F1: {round(f1_score(y_test, y_pred), 3) * 100}%")

Přesnost F1: 87.7%


Skoro 88% úspěšnost s random forest algoritmem, to je velmi hezké na to, že jsme nepoužili žádné ladění.

Jako 3. algoritmus se zvolíme Multi-layer-perceptron (MLP)

In [54]:
perceptron_classifier = MLPClassifier(random_state=25)
perceptron_classifier.fit(x_train, y_train)

y_pred = perceptron_classifier.predict(x_test)

print(f"Přenost F1: {round(f1_score(y_test, y_pred), 3) * 100}%")

Přenost F1: 87.8%


C:\Users\Tobias\PycharmProjects\ZSU\.venv\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Opět skoro 88% úspěšnost, stejně jako s random forest algoritmem.

Nyní půjdeme na K-fold validaci, kde použijeme normální rozdělení, protože náš dataset je rozdělen na 45/55, což je skoro rovnoměrně rozdělené, takže není potřeba používat stratifikovanou verzi, která slouží hlavně v případech, kdy dataset není vyvážený.



In [55]:
models = {
    "Decision Tree": DecisionTreeClassifier(random_state=25),
    "Random Forest": RandomForestClassifier(random_state=25),
    "MLP Classifier": MLPClassifier(random_state=25)
}

kf = KFold(n_splits=5, shuffle=True, random_state=25)
scorer = make_scorer(f1_score)

results = []

for name, model in models.items():
    scores = cross_val_score(model, X, y, cv=kf, scoring=scorer)
    results.append({
        "Algorithm": name,
        "Mean F1 Score (in %)": (round(scores.mean(), 3) * 100),
        "Std F1 Score": scores.std()
    })

results_df = pd.DataFrame(results)
print(results_df)


C:\Users\Tobias\PycharmProjects\ZSU\.venv\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Tobias\PycharmProjects\ZSU\.venv\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Tobias\PycharmProjects\ZSU\.venv\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Tobias\PycharmProjects\ZSU\.venv\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


        Algorithm  Mean F1 Score (in %)  Std F1 Score
0   Decision Tree                  80.4      0.028688
1   Random Forest                  89.1      0.007141
2  MLP Classifier                  87.3      0.006359


C:\Users\Tobias\PycharmProjects\ZSU\.venv\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Jak můžeme vidět, Nejlepší výsledky má random forest s nejlepší přesností, tudíž se ho pokusíme vytunit co nejvíce v následující částí, kdy nám půjde hlavně o tuning parametrů.

Nejdřív ale potuníme parametry i u decision tree a MLP classifieru a random forest si necháme na závěr, jelikož dává nejlepší výsledky.

Pro tuning využiji knihovny GridSearchCV, která za nás projde algoritmus s různými parametry

In [56]:
results_tree = []

param_grid_tree = {
    "max_depth": [3, 5, 7, 10, None],
    "min_samples_split": [2, 5, 10, 20]
}

grid_tree = GridSearchCV(
    DecisionTreeClassifier(random_state=25),
    param_grid_tree,
    cv=5,
    scoring="f1",
    n_jobs=-1
)

grid_tree.fit(x_train, y_train)

for params, mean_score in zip(grid_tree.cv_results_["params"], 
                            grid_tree.cv_results_["mean_test_score"]):
    results_tree.append({
        "Algorithm": "Decision Tree",
        "Parameters": f'max_depth={params["max_depth"]}, min_samples_split={params["min_samples_split"]}',
        "F1 Score": f"{round(mean_score * 100, 2)}%"
    })
    
results_tree = pd.DataFrame(results_tree)

print(results_tree)
print("\nNejlepší výsledek:\n", results_tree.max())

        Algorithm                            Parameters F1 Score
0   Decision Tree      max_depth=3, min_samples_split=2   83.39%
1   Decision Tree      max_depth=3, min_samples_split=5   83.39%
2   Decision Tree     max_depth=3, min_samples_split=10   83.39%
3   Decision Tree     max_depth=3, min_samples_split=20   83.39%
4   Decision Tree      max_depth=5, min_samples_split=2    83.8%
5   Decision Tree      max_depth=5, min_samples_split=5   83.66%
6   Decision Tree     max_depth=5, min_samples_split=10   83.78%
7   Decision Tree     max_depth=5, min_samples_split=20   84.51%
8   Decision Tree      max_depth=7, min_samples_split=2   82.74%
9   Decision Tree      max_depth=7, min_samples_split=5   82.62%
10  Decision Tree     max_depth=7, min_samples_split=10   82.78%
11  Decision Tree     max_depth=7, min_samples_split=20   84.03%
12  Decision Tree     max_depth=10, min_samples_split=2   79.33%
13  Decision Tree     max_depth=10, min_samples_split=5   79.99%
14  Decision Tree    max_

Když si seřadíme tabulku pro F1 score, tak vidíme, že Decision tree se dostal na 84.5%, což je 4% zlepšení oproti algoritmu bez tuningu

Nyní se půjdeme podívat stejným způsobem na MLP classifier

In [57]:
results_mlp = []

param_grid_mlp = {
    "hidden_layer_sizes": [(50,), (100,), (50, 50), (100, 50)],
    "activation": ["relu", "tanh"],
    "learning_rate_init": [0.001, 0.01],
    "max_iter": [10, 100, 500, 1000],
}

grid_mlp = GridSearchCV(
    MLPClassifier(random_state=25),
    param_grid_mlp,
    cv=5,
    scoring="f1",
    n_jobs=-1
)

grid_mlp.fit(x_train, y_train)

for params, mean_score in zip(grid_mlp.cv_results_["params"], 
                            grid_mlp.cv_results_["mean_test_score"]):
    results_mlp.append({
        "Algorithm": "MLP",
        "Parameters": f'layers={params["hidden_layer_sizes"]}, activation={params["activation"]}, lr={params["learning_rate_init"]}, max_iter={params["max_iter"]}',
        "F1 Score": f"{round(mean_score * 100, 2)}%"
    })
    
results_mlp = pd.DataFrame(results_mlp)

print(results_mlp)
print("\nNejlepší výsledek:\n", results_mlp.max())

   Algorithm                                         Parameters F1 Score
0        MLP  layers=(50,), activation=relu, lr=0.001, max_i...   85.05%
1        MLP  layers=(50,), activation=relu, lr=0.001, max_i...   87.23%
2        MLP  layers=(50,), activation=relu, lr=0.001, max_i...   87.25%
3        MLP  layers=(50,), activation=relu, lr=0.001, max_i...   86.45%
4        MLP  layers=(50,), activation=relu, lr=0.01, max_it...   87.12%
..       ...                                                ...      ...
59       MLP  layers=(100, 50), activation=tanh, lr=0.001, m...   85.41%
60       MLP  layers=(100, 50), activation=tanh, lr=0.01, ma...   86.33%
61       MLP  layers=(100, 50), activation=tanh, lr=0.01, ma...    86.7%
62       MLP  layers=(100, 50), activation=tanh, lr=0.01, ma...   84.83%
63       MLP  layers=(100, 50), activation=tanh, lr=0.01, ma...   84.83%

[64 rows x 3 columns]

Nejlepší výsledek:
 Algorithm                                                   MLP
Parameters    la

C:\Users\Tobias\PycharmProjects\ZSU\.venv\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(


MLP classifier se nám nepodařilo moc vytáhnout, oproti originálním 87.3% se nám podařilo dosáhnout 88.2%, což je zlepšení o necelé 1 %.

Nyní zkusíme ale vylepšit samotný random forest a jelikož vykazoval nejlepší úspěšnost, zkusíme opravdu hodně parametrů a budeme doufat v dobré výsledky.

In [58]:
results_random = []

param_grid_rf = {
    "n_estimators": [100, 200, 300],
    "max_depth": [5, 10, None],
    "min_samples_split": [2, 5, 10],
    "min_samples_leaf": [1, 2, 4],
    "max_features": ["sqrt", "log2"],
    "criterion": ["gini", "entropy"],
}

grid_rf = GridSearchCV(
    RandomForestClassifier(random_state=25),
    param_grid_rf,
    cv=5,
    scoring="f1",
    n_jobs=-1
)

grid_rf.fit(x_train, y_train)

for params, mean_score in zip(grid_rf.cv_results_["params"], 
                            grid_rf.cv_results_["mean_test_score"]):
    results_random.append({
        "Algorithm": "Random Forest",
        "Parameters": f'trees={params["n_estimators"]}, depth={params["max_depth"]}, '
                     f'min_split={params["min_samples_split"]}, min_leaf={params["min_samples_leaf"]}, '
                     f'max_feat={params["max_features"]}, crit={params["criterion"]}',
        "F1 Score": f"{round(mean_score * 100, 2)}%"
    })
    
results_random = pd.DataFrame(results_random)

print(results_random)
print("\nNejlepší výsledek:\n", results_random.max())

C:\Users\Tobias\PycharmProjects\ZSU\.venv\Lib\site-packages\numpy\ma\core.py:2881: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


         Algorithm                                         Parameters F1 Score
0    Random Forest  trees=100, depth=5, min_split=2, min_leaf=1, m...   89.04%
1    Random Forest  trees=200, depth=5, min_split=2, min_leaf=1, m...   89.07%
2    Random Forest  trees=300, depth=5, min_split=2, min_leaf=1, m...    89.2%
3    Random Forest  trees=100, depth=5, min_split=5, min_leaf=1, m...   89.47%
4    Random Forest  trees=200, depth=5, min_split=5, min_leaf=1, m...    89.6%
..             ...                                                ...      ...
319  Random Forest  trees=200, depth=None, min_split=5, min_leaf=4...   89.76%
320  Random Forest  trees=300, depth=None, min_split=5, min_leaf=4...   89.25%
321  Random Forest  trees=100, depth=None, min_split=10, min_leaf=...    89.6%
322  Random Forest  trees=200, depth=None, min_split=10, min_leaf=...   89.73%
323  Random Forest  trees=300, depth=None, min_split=10, min_leaf=...   89.39%

[324 rows x 3 columns]

Nejlepší výsledek:
 Algorit

Tady ale se už taktéž bohužel moc neposuneme, oproti původním 88% je to posun o 2%. Nakonec ale vychází random forest lépe a s 90+% úspěšností mi to příjde jako velmi hezký úspěch na velikost datasetu.

Na závěr spojíme ještě všechny tabulky do sebe, ať můžeme vidět nejúspěšnější algoritmy a nastavení, pokud bychom chtěli.

In [60]:
all_results = pd.concat([results_tree, results_random, results_mlp], ignore_index=True)

all_results

,Algorithm,Parameters,F1 Score
0,Decision Tree,"max_depth=3, min_samples_split=2",83.39%
1,Decision Tree,"max_depth=3, min_samples_split=5",83.39%
2,Decision Tree,"max_depth=3, min_samples_split=10",83.39%
3,Decision Tree,"max_depth=3, min_samples_split=20",83.39%
4,Decision Tree,"max_depth=5, min_samples_split=2",83.8%
...,...,...,...
403,MLP,"layers=(100, 50), activation=tanh, lr=0.001, m...",85.41%
404,MLP,"layers=(100, 50), activation=tanh, lr=0.01, ma...",86.33%
405,MLP,"layers=(100, 50), activation=tanh, lr=0.01, ma...",86.7%
406,MLP,"layers=(100, 50), activation=tanh, lr=0.01, ma...",84.83%


ZÁVĚR

Závěrem bych řekl, že naše motivace naučit se rozpoznávat to, jestli pacient má nebo nemá srdeční chorobu na základě jeho hodnot naměřených byla celkem úspěšná. 

Nejdřív jsme si prošli samotná data, s kterými pracujeme, provedli jejich úpravu, která byla relativně snadná a data nevyžadovala moc práce na předělání.

Protože se jedná o medicínská data a jsou důležití falešně pozitivní i falešně negativní, tak jsem použil F1 score jako metriku vyhodnocování úspěšnosti algoritmu.

Poté jsme si zkusili bez jakýchkoliv parametrů použít decision tree, random forest a na závěr MLP classifier. Z těchto výsledků bylo rychle vidět, že random forest i MLP měli podobné výsledky, ale decision tree dost zaostával.

Následně jsme přešli k-fold cross validaci, kde jsme si ověřili, jestli algoritmy na daném datasetu opravdu fungují dobře a jestli nejsou moc přetrénovaná, nebo podtrénovaná.

Poté jsme přešli k poslední části a to tuningu parametrů. Tady jsem upřímně řečeno měl velké naděje pro random forest, vzhledem k tomu, jak dobře vycházel bez parametrů. Avšak po tuningu parametrů se ukázalo razantní zlepšení pouze u decision tree, MLP se posunul o 1 procento a random forest o procenta 2.

Na závěr se nám povedlo najít parametry pro random forest, který dosahoval 90+% úspěšnosti, což považuji za dost dobré vzhledem k velikosti datasetu. 

Upřímně mi přišla práce s datasetem dost jednoduchá a intuitivní, přimočará a neměl jsem žádné problémy.

Tobias Janča
JAN0895